In [1]:
%matplotlib inline
import os
import mne
import pandas as pd
from mne.externals.pymatreader import read_mat
import numpy as np
import matplotlib.pyplot as plt
import itertools
from glob import glob
import pyriemann
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score 
import time

In [2]:
cwd = os.getcwd()
data_path = cwd
n_subs = 4
n_sessions = 2
diff = ['MATBeasy', 'MATBmed', 'MATBdiff']

In [3]:
electrodes = pd.read_csv(data_path + '/chan_locs_standard',header=None, sep ='\t', names=['ch_names','x','y','z'])
electrodes.head()

,ch_names,x,y,z
0,Fp1,-29.4370,83.917,-6.990
1,Fz,0.3122,58.512,66.462
2,F3,-50.2440,53.111,42.192
3,F7,-70.2630,42.474,-11.420
4,FT9,-84.0760,14.567,-50.429


Covariance estimation For robust covariance estimation, we take advantage of shrinkage. Here
the Oracle Approximating Shrinkage (OAS) is used. #### Classifier We use a simple Logistic
Regression (with a non-optimized L2 penalty) on Tangent Space Features, extracted with Pyriemann
toolbox. #### Channel selection A manual and naive EEG channel selection is performed
to use 13 electrodes, mostly frontal.

In [4]:
lr = LogisticRegression(C=1/10.)
clf = make_pipeline(pyriemann.estimation.Covariances(estimator='oas'), pyriemann.classification.TSclassifier(clf=lr))
ch_slice = ['F7', 'F5', 'F3', 'F1', 'F2', 'F4', 'F6', 'AF3', 'AFz', 'AF4', 'FP1', 'FP2', 'FPz']

In [6]:
# Single Subj epochs classification
for sub_n, session_n in itertools.product(range(n_subs), range(n_sessions)):
    epochs_data = []
    labels = []
    for lab_idx, level in enumerate(diff):
        sub = 'P{0:02d}'.format(sub_n+1)
        sess = f'S{session_n+1}'
        path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
#         print(path)
        # Read the epoched data with MNE
        epochs = mne.io.read_epochs_eeglab(path, verbose=False)
        # You could add some pre-processing here with MNE
        # We will just select some channels (mostly frontal ones)
        epochs = epochs.drop_channels(list(set(epochs.ch_names) - set(ch_slice)))
        # Get the data and concatenante with others MATB levels
        tmp = epochs.get_data()
        epochs_data.extend(tmp)
        labels.extend([lab_idx]*len(tmp))

    epochs_data = np.array(epochs_data)
    labels = np.array(labels)

    # Compute classification accuracy with 5-folds cross validation
    start = time.time()
    acc = cross_val_score(clf, X=epochs_data, y=labels, cv=5)
    end = time.time()
    print(f'time = {end-start}')
    print(f'Subject {sub} and session {session_n+1}: mean accuracy of {round(np.mean(acc), 2)} with a standard deviation of {round(np.std(acc), 2)}')

time = 3.9678661823272705
Subject P01 and session 1: mean accuracy of 0.78 with a standard deviation of 0.03
time = 3.8175976276397705
Subject P01 and session 2: mean accuracy of 0.74 with a standard deviation of 0.06
time = 3.891139030456543
Subject P02 and session 1: mean accuracy of 0.74 with a standard deviation of 0.04
time = 3.886229991912842
Subject P02 and session 2: mean accuracy of 0.83 with a standard deviation of 0.08
time = 3.8696224689483643
Subject P03 and session 1: mean accuracy of 0.67 with a standard deviation of 0.05
time = 3.850278854370117
Subject P03 and session 2: mean accuracy of 0.63 with a standard deviation of 0.1
time = 3.811692237854004
Subject P04 and session 1: mean accuracy of 0.73 with a standard deviation of 0.03
time = 3.878727912902832
Subject P04 and session 2: mean accuracy of 0.78 with a standard deviation of 0.06


In [7]:
#Transfer from senssion 1 to 2 for P01
sub_n = 0
session_n = 0
epochs_data = []
labels = []
for lab_idx, level in enumerate(diff):
    sub = 'P{0:02d}'.format(sub_n+1)
    sess = f'S{session_n+1}'
    path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
    # Read the epoched data with MNE
    epochs = mne.io.read_epochs_eeglab(path, verbose=False)
    # You could add some pre-processing here with MNE
    # We will just select some channels (mostly frontal ones)
    epochs = epochs.drop_channels(list(set(epochs.ch_names) - set(ch_slice)))
    # Get the data and concatenante with others MATB levels
    tmp = epochs.get_data()
    epochs_data.extend(tmp)
    labels.extend([lab_idx]*len(tmp))
epochs_data = np.array(epochs_data)
labels = np.array(labels)
# Train the model on all epochs from session 1
start = time.time()
clf.fit(epochs_data, labels)
end = time.time()
print(f'Fitting time on Subject {sub_n+1} Sess {session_n+1} = {end-start}')

Fitting time on Subject 1 Sess 1 = 0.9518132209777832


In [8]:
session_n = 1
epochs_data = []
labels = []
for lab_idx, level in enumerate(diff):
    sub = 'P{0:02d}'.format(sub_n+1)
    sess = f'S{session_n+1}'
    path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
    # Read the epoched data with MNE
    tmp = mne.io.read_epochs_eeglab(path, verbose=False)
    # You could add some pre-processing here with MNE
    # We will just select some channels (mostly frontal ones)
    epochs = epochs.drop_channels(list(set(epochs.ch_names) - set(ch_slice)))
    # Get the data and concatenante with others MATB levels
    tmp = epochs.get_data()
    epochs_data.extend(tmp)
    labels.extend([lab_idx]*len(tmp))
epochs_data = np.array(epochs_data)
labels = np.array(labels)
# Use trained model to predict for all epochs of session 2 and compute accuracy
start = time.time()
y_pred = clf.predict(epochs_data)
end = time.time()
print(f'Prediction time on Subject {sub_n+1} Sess {session_n+1} = {end-start}')
acc = accuracy_score(labels, y_pred)
print(f'Subject {sub} and transfer from session 1 to 2: mean accuracy of {round(acc, 2)}.')

Prediction time on Subject 1 Sess 2 = 0.14130353927612305
Subject P01 and transfer from session 1 to 2: mean accuracy of 0.33.


In [ ]:
#Generate a CSV to submit
submission = pd.DataFrame({'epochID':np.arange(len(y_pred)), 'prediction' : y_pred})
submission.head()
submission.to_csv("submission.csv",header=True,index=False)